# Agenda

1. Lots of questions
2. Even more questions!

# What are the methods that all Python objects have?

- Everything in Python is an object
- Everything has a class (type)
- Everything has attributes as well (things that come after a .)

- Methods are attributes
- Through inheritance, objects have access to methods on their classes, and also anything that their class inherits from

- Everyone eventually inherits from `object`, the ultimate top object in the Python hierarchy.

So the only methods that *everything* in Python has access to are those that are defined on `object`, and which we then get via inheritance.

What are those?

- `__new__` -- this creates new objects.  You should *NOT EVER EVER EVER* define this method unless you *really* know what you're doing.
- `__init__` -- this adds new attributes to objecst. This is typically overridden in a class you create.
- `__str__` -- this returns a string representation of your object, meant for end users
- `__repr__` -- this returns a string representation of your object, meant for programmers.

There are a few other methods as well, such as `__setattr__` and `__getattr__` which are kind of low level.